In [36]:
import json
def format_answer(answer):
    if answer == "entailment":
        return "yes"
    elif answer == "contradiction":
        return "no"
    else:
        return "unknown"
f = open("snli_1.0_test.jsonl", "r")
temp = {i : json.loads(line) for i, line in enumerate(f)}
i=1
for k, v in temp.items():
    sentence1 = v["sentence1"]
    sentence2 = v["sentence2"]
    answer = format_answer(v["gold_label"])
    if len(sentence1.split()) < 15 and len(sentence2.split()) < 15:
        g = open("select_snli/sick_test_"+str(i)+".txt", "w")
        g.write(sentence1+"\n")
        g.write(sentence2)
        g.close()
        h = open("select_snli/sick_test_"+str(i)+".answer", "w")
        h.write(answer+"\n")
        h.close()
        i+=1

In [79]:
import glob
files = glob.glob("plain/sick_*.txt")
sentence1, sentence2 = [], []
for file in files:
    f = open(file, "r")
    sentences = f.readlines()
    f.close()
    sentence1.append(sentences[0].strip())
    sentence2.append(sentences[0])
sentence = sentence1 + sentence2

In [101]:
#Calculate PMI for validity of phrases
from gensim import models
import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

stopset = set(stopwords.words('english')+[","])
tokens = word_tokenize("\t".join(sentence))
tokens = [w.lower() for w in tokens if not w in stopset]
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(5)
scores = [l[1] for l in finder.score_ngrams(bigram_measures.pmi)]
meanscores = sum(scores)/len(scores)
for i in finder.score_ngrams(bigram_measures.pmi):
    if i[1] > meanscores:
        print(i)

(('bedroom', 'slipper'), 14.46263007478904)
(('dueling', 'sticks'), 14.46263007478904)
(('dusty', 'farm'), 14.46263007478904)
(('failing', 'perform'), 14.46263007478904)
(('probably', 'pine'), 14.46263007478904)
(('reserved', 'handicapped'), 14.46263007478904)
(('sneezing', 'scaring'), 14.46263007478904)
(('space', 'reserved'), 14.46263007478904)
(('twin', 'preteen'), 14.46263007478904)
(('dangling', 'straps'), 14.047592575510194)
(('extended', 'outward'), 14.047592575510194)
(('revealing', 'extravagant'), 14.047592575510194)
(('saying', 'word'), 14.047592575510194)
(('substance', 'blurry'), 14.047592575510194)
(('tiny', 'bit'), 14.047592575510194)
(('waste', 'bin'), 14.047592575510194)
(('buttered', 'loaves'), 13.725664480622832)
(('experiencing', 'new'), 13.725664480622832)
(('fancy', 'dresses'), 13.725664480622832)
(('filling', 'pitcher'), 13.725664480622832)
(('masonry', 'pyramid'), 13.725664480622832)
(('new', 'world'), 13.725664480622832)
(('scratching', 'belly'), 13.725664480622

In [1]:
import json
import glob
import re
import os
import numpy as np
import scipy as sp
import argparse
import random
import difflib

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing, linear_model, svm
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.cross_validation import cross_val_score

with open('phrase_classifier_23fd213/features.pickle', 'rb') as in_f:
    load_target = np.load(in_f)
    load_source = np.load(in_f)
    load_source_phrase = np.load(in_f)

def crossvalidation(clf, X_train, y_train):
    scores = cross_val_score(clf, X_train, y_train, cv=10)
    return scores.mean(), scores.std()

def classification(X_train, y_train, X_test, y_test, results=None):
    parameters = {
        'n_estimators'      : [10, 50, 100, 200, 300, 400, 500],
        'random_state'      : [0],
        'n_jobs'            : [200],
        'max_features'      : ['auto', 'log2', 'sqrt', None],
        'criterion'         : ['gini'],
        'max_depth'         : [3, 5, 10, 20, 30, 40, 50, 100]
    }
    #random forest
    #clf = make_pipeline(
    #    preprocessing.StandardScaler(),
    #    GridSearchCV(RandomForestClassifier(), parameters))
    #clf = svm.SVC(C=1.0, gamma=0.001)
    clf = svm.SVC(kernel='linear') #SVM kernel='linear'の方が良い様子
    
    clf.fit(X_train, y_train)

    #Serialize
    #joblib.dump(clf, './'+results+'/phrase.pkl')
    #clf = joblib.load('./'+results+'/phrase.pkl')

    return clf

/Users/yanakahitomi/.pyenv/versions/3.5.0/envs/ccg2lambda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yanakahitomi/.pyenv/versions/3.5.0/envs/ccg2lambda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
half = int(len(load_target)/2)
train_sources, trial_sources = load_source[:half], load_source[half:]
train_targets, trial_targets = load_target[:half], load_target[half:]
train_phrases, trial_phrases = load_source_phrase[:half], load_source_phrase[half:]
print ('test size: {0}, training size: {1}'.format(len(trial_targets), len(train_targets)))
    
# Train the regressor
clf = classification(train_sources, train_targets, trial_sources, trial_targets)

# predict
outputs = clf.predict(trial_sources)

#負例に対して正例が少なすぎる問題
trial_targets = list(map(str, trial_targets))
outputs = list(map(str, list(outputs)))
print(classification_report(trial_targets, outputs, digits=4))

test size: 52267, training size: 52266


             precision    recall  f1-score   support

        0.0     0.6401    0.9339    0.7596     32328
        1.0     0.5811    0.1486    0.2367     19939

avg / total     0.6176    0.6343    0.5601     52267

